In [77]:
import os.path
from os import path
import src.reader as Reader
import shutil
import yaml
import subprocess

# YamlConfig = 'yaml/Binomial20221007RBF_N4.yaml'
YamlConfig = 'yaml/Binomial20221007FixedRBF_N4.yaml'

In [78]:
# Check if all the prediction files are present
with open(YamlConfig, "r") as stream:
    Setup = yaml.safe_load(stream)

for Item in Setup['Data']:
    if path.isfile(Setup['BaseDirectory'] + '/' + Setup['Data'][Item]['Prediction']) == False:
        print('File not found: ' + Setup['Data'][Item]['Prediction'])
    if path.isfile(Setup['BaseDirectory'] + '/' + Setup['Data'][Item]['PredictionError']) == False:
        print('File not found: ' + Setup['Data'][Item]['Prediction'])

print("Check finished.  If nothing shows up it's ok.")

Check finished.  If nothing shows up it's ok.


In [79]:
# Check number of points in data vs calculation & error
with open(YamlConfig, "r") as stream:
    Setup = yaml.safe_load(stream)

for Item in Setup['Data']:
    Data = Reader.ReadData(Setup['BaseDirectory'] + Setup['Data'][Item]['Data'])
    Prediction = Reader.ReadPrediction(Setup['BaseDirectory'] + Setup['Data'][Item]['Prediction'])
    PredictionError = Reader.ReadPrediction(Setup['BaseDirectory'] + Setup['Data'][Item]['PredictionError'])
    
    DataPoints = Data['Data']['x'].shape[0]
    if 'DataExclude' in Setup['Data'][Item]:
        DataPoints = DataPoints - len(Setup['Data'][Item]['DataExclude'])
        
    PredictionPoints = Prediction['Prediction'].shape[1]
    if 'PredictionExclude' in Setup['Data'][Item]:
        PredictionPoints = PredictionPoints - len(Setup['Data'][Item]['PredictionExclude'])
    
    PredictionErrorPoints = PredictionError['Prediction'].shape[1]
    if 'PredictionExclude' in Setup['Data'][Item]:
        PredictionErrorPoints = PredictionErrorPoints - len(Setup['Data'][Item]['PredictionExclude'])
    
    if DataPoints != PredictionPoints or DataPoints != PredictionErrorPoints:
        print(f'{Item}: Data = {DataPoints}, Prediction = {PredictionPoints}, Error = {PredictionErrorPoints}')

print("Check finished.  If nothings shows up it's ok.")

Check finished.  If nothings shows up it's ok.


In [80]:
# First check if number of design points match between calculation and error
with open(YamlConfig, "r") as stream:
    Setup = yaml.safe_load(stream)

for Item in Setup['Data']:
    Prediction = Reader.ReadPrediction(Setup['BaseDirectory'] + Setup['Data'][Item]['Prediction'])
    PredictionError = Reader.ReadPrediction(Setup['BaseDirectory'] + Setup['Data'][Item]['PredictionError'])
    
    PredictionPoints = Prediction['Prediction'].shape[0]
    PredictionErrorPoints = PredictionError['Prediction'].shape[0]
    
    if PredictionPoints != PredictionErrorPoints:
        print(f'{Item}: Prediction has {PredictionPoints} design points but PredictionError has {PredictionErrorPoints}')

print("Check finished.  If nothings shows up it's ok.")

Check finished.  If nothings shows up it's ok.


In [81]:
# Now check if number of design points match across items
with open(YamlConfig, "r") as stream:
    Setup = yaml.safe_load(stream)

MaxDesign = 0

for Item in Setup['Data']:
    Prediction = Reader.ReadPrediction(Setup['BaseDirectory'] + Setup['Data'][Item]['Prediction'])
    if MaxDesign < Prediction['Prediction'].shape[0]:
        MaxDesign = Prediction['Prediction'].shape[0]

print(f'Max number of design points is {MaxDesign}')

Design = Reader.ReadDesign(Setup['BaseDirectory'] + Setup['Design']['File'])

TotalDesign = Design['Design'].shape[0]

print(f'Number of design points in the design file is {TotalDesign}')

Changed = False

for Item in Setup['Data']:
    Prediction = Reader.ReadPrediction(Setup['BaseDirectory'] + Setup['Data'][Item]['Prediction'])

    GuessedList = []
    with open(Setup['BaseDirectory'] + Setup['Data'][Item]['Prediction']) as f:
        for Line in f:
            if 'design_point' not in Line:
                continue
            Line = Line.replace('design_point', '')
                
            Items = Line.split()
            if(len(Items) < 2):
                continue
            if Items[0] != '#':
                continue
                
            GuessedList = [int(i) for i in Items if i != '#']
    
    if 'PredictionMissing' in Setup['Data'][Item]:
        for i in Setup['Data'][Item]['PredictionMissing']:
            if i in GuessedList:
                print(f'{Item}: Design point {i} is in both the header and the PredictionMissing list')
    
    # print(Item, GuessedList, PredictionMissing)
    
    CorrectPrediction = [i for i in range(TotalDesign) if i not in GuessedList]
    
    if len(CorrectPrediction) == 0:
        if 'PredictionMissing' in Setup['Data'][Item]:
            print(f'{Item}: nothing is missing, please remove the PredictionMissing line')
            Setup['Data'][Item].pop('PredictionMissing')
            Changed = True
    else:
        if 'PredictionMissing' not in Setup['Data'][Item] or CorrectPrediction != Setup['Data'][Item]['PredictionMissing']:
            print(f'{Item}: please update the list of missing prediction to')
            print(f'    PredictionMissing: {CorrectPrediction}')
            Setup['Data'][Item]['PredictionMissing'] = CorrectPrediction
            Changed = True

if Changed == True:
    print("Check finished.  Some changes are recommended.  Saved to temp.yaml")
    with open(r'temp.yaml', 'w') as file:
        yaml.safe_dump(Setup, file, indent=2, allow_unicode=False, default_flow_style=None, line_break=None)

else:
    print("Check finished.  If nothings shows up it's ok.")



Max number of design points is 177
Number of design points in the design file is 200
Check finished.  If nothings shows up it's ok.
